In [3]:
from sqlalchemy import MetaData, Table, Column, String, Integer

metadata = MetaData()
user_table = Table(
    "user_account",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("username", String(50)),
    Column("fullname", String(50)),
)

from sqlalchemy import create_engine

engine = create_engine("sqlite://", echo=True)
with engine.begin() as conn:
    metadata.create_all(conn)

2023-06-23 21:12:49,755 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-23 21:12:49,756 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2023-06-23 21:12:49,756 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-23 21:12:49,757 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2023-06-23 21:12:49,757 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-23 21:12:49,758 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	username VARCHAR(50), 
	fullname VARCHAR(50), 
	PRIMARY KEY (id)
)


2023-06-23 21:12:49,758 INFO sqlalchemy.engine.Engine [no key 0.00030s] ()
2023-06-23 21:12:49,759 INFO sqlalchemy.engine.Engine COMMIT


In [4]:
user_table.c.username

Column('username', String(length=50), table=<user_account>)

In [5]:
user_table.c.username == "spongebob"

In [11]:
x = user_table.c.username == "spongebob"
x.left

Column('username', String(length=50), table=<user_account>)

In [8]:
x.right

BindParameter('%(140641058080368 username)s', 'spongebob', type_=String(length=50))

In [9]:
x.operator

<function _operator.eq(a, b, /)>

In [15]:
# username_1 is called a bind parameter.
str(user_table.c.username == "spongebob")

'user_account.username = :username_1'

In [18]:
#Secure concern: values are passed by parameters form.
compiled = (user_table.c.username == "spongebob").compile()
print(compiled.string)
print(compiled.params)

user_account.username = :username_1
{'username_1': 'spongebob'}


In [23]:
from sqlalchemy import and_, or_

# And | Or
print(
    or_(
        user_table.c.username == "spongebob",
        user_table.c.username == "patrick"
    )
)

#Composition.
print(
    and_(
        user_table.c.fullname == "spongebob squarepants",
        or_(
            user_table.c.username == "spongebob",
            user_table.c.username == "patrick",
        )
    )
)

# More operators.
print(and_(
    user_table.c.id >= 5,
    user_table.c.fullname.between('m', 'z'),
    user_table.c.fullname != 'plankton'
))

# None will produce NULL.
print(and_(
    user_table.c.username != None,
    user_table.c.fullname == None
))

# Type sensitive, + can mean string concatenation.
print(user_table.c.id + 5)
print(user_table.c.fullname + " Jr.")


user_account.username = :username_1 OR user_account.username = :username_2
user_account.fullname = :fullname_1 AND (user_account.username = :username_1 OR user_account.username = :username_2)
user_account.id >= :id_1 AND user_account.fullname BETWEEN :fullname_1 AND :fullname_2 AND user_account.fullname != :fullname_3
user_account.username IS NOT NULL AND user_account.fullname IS NULL
user_account.id + :id_1
user_account.fullname || :fullname_1


In [25]:
criteria = user_table.c.username.in_(["sandy", "squidward", "spongebob"])
print(criteria)
print(criteria.compile(compile_kwargs={'render_postcompile': True}))

criteria = user_table.c.username.in_([])
print(criteria.compile(compile_kwargs={'render_postcompile': True}))

user_account.username IN (__[POSTCOMPILE_username_1])
user_account.username IN (:username_1_1, :username_1_2, :username_1_3)
user_account.username IN (NULL) AND (1 != 1)


In [27]:
insert_stmt = user_table.insert().values(
    username="spongebob", fullname="Spongebob Squarepants"
)

with engine.begin() as conn:
    conn.execute(insert_stmt)

2023-06-23 21:40:47,273 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-23 21:40:47,274 INFO sqlalchemy.engine.Engine INSERT INTO user_account (username, fullname) VALUES (?, ?)
2023-06-23 21:40:47,274 INFO sqlalchemy.engine.Engine [cached since 10.24s ago] ('spongebob', 'Spongebob Squarepants')
2023-06-23 21:40:47,275 INFO sqlalchemy.engine.Engine COMMIT


In [28]:
# Alternative way of inserting.
with engine.begin() as conn:
    # Single row.
    conn.execute(
        user_table.insert(), {"username": "sandy", "fullname": "Sandy Cheeks"}
    )

    # List of dicts will be multiple rows.
    conn.execute(
        user_table.insert(),
        [
            {"username": "patrick", "fullname": "Patrick Star"},
            {"username": "squidward", "fullname": "Squidward Tentacles"},
        ],
    )

2023-06-23 21:44:57,341 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-23 21:44:57,342 INFO sqlalchemy.engine.Engine INSERT INTO user_account (username, fullname) VALUES (?, ?)
2023-06-23 21:44:57,342 INFO sqlalchemy.engine.Engine [generated in 0.00067s] ('sandy', 'Sandy Cheeks')
2023-06-23 21:44:57,343 INFO sqlalchemy.engine.Engine INSERT INTO user_account (username, fullname) VALUES (?, ?)
2023-06-23 21:44:57,343 INFO sqlalchemy.engine.Engine [generated in 0.00031s] [('patrick', 'Patrick Star'), ('squidward', 'Squidward Tentacles')]
2023-06-23 21:44:57,344 INFO sqlalchemy.engine.Engine COMMIT


In [29]:
from sqlalchemy import select

with engine.connect() as connection:
    select_stmt = (
        select(user_table.c.username, user_table.c.fullname).
        where(user_table.c.username == "spongebob")
    )

    result = connection.execute(select_stmt)
    for row in result:
        print(row)

2023-06-24 12:38:23,060 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-24 12:38:23,061 INFO sqlalchemy.engine.Engine SELECT user_account.username, user_account.fullname 
FROM user_account 
WHERE user_account.username = ?
2023-06-24 12:38:23,061 INFO sqlalchemy.engine.Engine [generated in 0.00161s] ('spongebob',)
('spongebob', 'Spongebob Squarepants')
2023-06-24 12:38:23,062 INFO sqlalchemy.engine.Engine ROLLBACK


In [31]:
with engine.connect() as connection:
    select_stmt = select(user_table)
    print(connection.execute(select_stmt).all())

2023-06-24 12:43:25,526 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-24 12:43:25,526 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.username, user_account.fullname 
FROM user_account
2023-06-24 12:43:25,527 INFO sqlalchemy.engine.Engine [cached since 157.8s ago] ()
[(1, 'spongebob', 'Spongebob Squarepants'), (2, 'sandy', 'Sandy Cheeks'), (3, 'patrick', 'Patrick Star'), (4, 'squidward', 'Squidward Tentacles')]
2023-06-24 12:43:25,527 INFO sqlalchemy.engine.Engine ROLLBACK


In [32]:
with engine.connect() as connection:
    select_stmt = select(user_table).where(
        or_(
            user_table.c.username == "spongebob",
            user_table.c.username == "sandy",
        )
    ).order_by(user_table.c.username)
    print(connection.execute(select_stmt).all())

2023-06-24 12:49:53,981 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-24 12:49:53,982 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.username, user_account.fullname 
FROM user_account 
WHERE user_account.username = ? OR user_account.username = ? ORDER BY user_account.username
2023-06-24 12:49:53,982 INFO sqlalchemy.engine.Engine [generated in 0.00120s] ('spongebob', 'sandy')
[(2, 'sandy', 'Sandy Cheeks'), (1, 'spongebob', 'Spongebob Squarepants')]
2023-06-24 12:49:53,983 INFO sqlalchemy.engine.Engine ROLLBACK


In [33]:
with engine.connect() as connection:
    select_stmt = (
        select(user_table)
        .where(user_table.c.username == "spongebob")
        .where(user_table.c.fullname == "Spongebob Squarepants")
        .order_by(user_table.c.username)
    )
    print(connection.execute(select_stmt).all())

2023-06-24 12:52:14,046 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-24 12:52:14,047 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.username, user_account.fullname 
FROM user_account 
WHERE user_account.username = ? AND user_account.fullname = ? ORDER BY user_account.username
2023-06-24 12:52:14,047 INFO sqlalchemy.engine.Engine [generated in 0.00111s] ('spongebob', 'Spongebob Squarepants')
[(1, 'spongebob', 'Spongebob Squarepants')]
2023-06-24 12:52:14,048 INFO sqlalchemy.engine.Engine ROLLBACK


In [34]:
connection = engine.connect()

result = connection.execute(
    select(user_table.c.fullname).where(user_table.c.username == 'spongebob')
)
result.one()

2023-06-24 12:57:19,531 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-24 12:57:19,532 INFO sqlalchemy.engine.Engine SELECT user_account.fullname 
FROM user_account 
WHERE user_account.username = ?
2023-06-24 12:57:19,532 INFO sqlalchemy.engine.Engine [generated in 0.00117s] ('spongebob',)


('Spongebob Squarepants',)

In [35]:
# If results have more than one row, will raise error. 
result = connection.execute(
    select(user_table.c.fullname).order_by(user_table.c.username)
)
result.one()

2023-06-24 13:00:53,543 INFO sqlalchemy.engine.Engine SELECT user_account.fullname 
FROM user_account ORDER BY user_account.username
2023-06-24 13:00:53,543 INFO sqlalchemy.engine.Engine [generated in 0.00075s] ()


MultipleResultsFound: Multiple rows were found when exactly one was required

In [36]:
result = connection.execute(
    select(user_table).where(user_table.c.username == 'nonexistent')
)
result.one_or_none()

2023-06-24 13:03:38,324 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.username, user_account.fullname 
FROM user_account 
WHERE user_account.username = ?
2023-06-24 13:03:38,325 INFO sqlalchemy.engine.Engine [generated in 0.00086s] ('nonexistent',)


In [39]:
# Results can do column slicing.
result = connection.execute(
    select(user_table).order_by(user_table.c.username)
)

for fullname, username in result.columns("fullname", "username"):
    print(f"{fullname} {username}")

# Single column can be delivered without using rows.
result = connection.execute(
    select(user_table).order_by(user_table.c.username)
)

for fullname in result.scalars("fullname"):
    print(fullname)

2023-06-24 13:12:10,261 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.username, user_account.fullname 
FROM user_account ORDER BY user_account.username
2023-06-24 13:12:10,262 INFO sqlalchemy.engine.Engine [cached since 176.3s ago] ()
Patrick Star patrick
Sandy Cheeks sandy
Spongebob Squarepants spongebob
Squidward Tentacles squidward
2023-06-24 13:12:10,263 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.username, user_account.fullname 
FROM user_account ORDER BY user_account.username
2023-06-24 13:12:10,263 INFO sqlalchemy.engine.Engine [cached since 176.3s ago] ()
Patrick Star
Sandy Cheeks
Spongebob Squarepants
Squidward Tentacles


In [40]:
# Update operations like insert operations.
with engine.begin() as connection:
    update_stmt = (
        user_table.update()
        .values(fullname="Patrick Star")
        .where(user_table.c.username == "patrick")
    )

    result = connection.execute(update_stmt)

# Given parameters version. 
with engine.begin() as connection:
    update_stmt = (
        user_table.update()
        .where(user_table.c.username == "patrick")
    )

    result = connection.execute(update_stmt, {"fullname": "Patrick Star"})

2023-06-24 13:27:04,755 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-24 13:27:04,757 INFO sqlalchemy.engine.Engine UPDATE user_account SET fullname=? WHERE user_account.username = ?
2023-06-24 13:27:04,758 INFO sqlalchemy.engine.Engine [generated in 0.00062s] ('Patrick Star', 'patrick')
2023-06-24 13:27:04,758 INFO sqlalchemy.engine.Engine COMMIT
2023-06-24 13:27:04,759 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-24 13:27:04,759 INFO sqlalchemy.engine.Engine UPDATE user_account SET fullname=? WHERE user_account.username = ?
2023-06-24 13:27:04,760 INFO sqlalchemy.engine.Engine [generated in 0.00041s] ('Patrick Star', 'patrick')
2023-06-24 13:27:04,760 INFO sqlalchemy.engine.Engine COMMIT


In [41]:
# Can accommodate SQL expressions.
with engine.begin() as connection:
    update_stmt = (
        user_table.update().
        values(
            fullname=user_table.c.username + " " + user_table.c.fullname
        )
    )

    result = connection.execute(update_stmt)

2023-06-24 13:33:59,459 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-24 13:33:59,462 INFO sqlalchemy.engine.Engine UPDATE user_account SET fullname=(user_account.username || ? || user_account.fullname)
2023-06-24 13:33:59,463 INFO sqlalchemy.engine.Engine [generated in 0.00089s] (' ',)
2023-06-24 13:33:59,464 INFO sqlalchemy.engine.Engine COMMIT


In [42]:
# Deletion.
with engine.begin() as connection:
    delete_stmt = user_table.delete().where(user_table.c.username == "patrick")

    result = connection.execute(delete_stmt)

2023-06-24 13:37:25,593 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-24 13:37:25,595 INFO sqlalchemy.engine.Engine DELETE FROM user_account WHERE user_account.username = ?
2023-06-24 13:37:25,595 INFO sqlalchemy.engine.Engine [generated in 0.00056s] ('patrick',)
2023-06-24 13:37:25,596 INFO sqlalchemy.engine.Engine COMMIT
